In [1]:
import numpy as np 
import pandas as pd 
import os
import lightgbm
import optuna
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss

In [2]:
BASE_ROOT = "Complete Dataset/"

In [3]:
train = pd.read_csv(BASE_ROOT+'Phising_Training_Dataset.csv')
test = pd.read_csv(BASE_ROOT+'Phising_Testing_Dataset.csv')
submission = pd.read_csv(BASE_ROOT+'Phising_Sample_Submisson.csv')

In [4]:
train

,key,having_IP,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,12344,-1,1,1,1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,12345,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,12346,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,12347,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,12348,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8950,21294,1,-1,1,-1,1,1,1,1,-1,...,-1,-1,1,1,-1,-1,1,1,1,1
8951,21295,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
8952,21296,1,-1,1,1,1,-1,1,-1,-1,...,1,1,1,1,1,-1,1,0,1,-1
8953,21297,-1,-1,1,1,1,-1,-1,-1,1,...,-1,1,1,1,1,-1,1,1,1,-1


In [5]:
train['Result'] = train['Result'].replace(-1,0)
X= train.iloc[: , 1:-1]
y= train['Result']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.01, random_state=69)

In [7]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [8]:
def objective(trial, X, y, name='gladiator1'):
        
    params = {'max_depth':trial.suggest_int('max_depth', 5, 70),
              'n_estimators':2000000,
              'subsample': trial.suggest_uniform('subsample', 0.2, 1.0),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.2, 1.0),
              'learning_rate':trial.suggest_uniform('learning_rate', 0.007, 0.02),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 15, 40),
              'reg_lambda':trial.suggest_uniform('reg_lambda', 0.01, 50),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 0.01, 50),
              'min_child_samples':trial.suggest_int('min_child_samples', 5, 100),
              'num_leaves':trial.suggest_int('num_leaves', 10, 200),
              'n_jobs' : -1,
              'metric': 'binary_logloss',
              'boosting_type': 'gbdt',
              'max_bin':trial.suggest_int('max_bin', 300, 1000),
              'cat_smooth':trial.suggest_int('cat_smooth', 5, 100),
              'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 100)}

    model = LGBMClassifier(objective="binary", **params)
                  
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01, random_state=0)
    

    model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
              eval_metric=['binary_logloss'],
              early_stopping_rounds=300, 
              
              #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
              verbose=1)
    return log_loss(y_val,model.predict(X_val))

In [18]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial,X=X.values, y=y.values)
pred = study.optimize(func, n_trials=50)

In [20]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t'{key}': {value}")

In [9]:
params = {
        'task': 'prediction',
        'boosting_type': 'gbdt',
        'metric': 'binary_logloss',
        'learning_rate': 0.017764429656900442,
        'max_depth': 62,
        'num_leaves': 190, 
        'min_data_in_leaf': 37,
        'num_iterations': 2000000,
        'verbosity': -1,
        'device':'cpu',
        'subsample':0.9988361911191573,
        'colsample_bytree':0.8638322535165064,
        'reg_lambda':19.708332913135084,
        'reg_alpha':0.0541168857042065,
        'min_child_samples':84,
        'max_bin': 728,
        'cat_smooth':58,
        'cat_l2':0.001571519620397213
    
}


In [10]:
model = LGBMClassifier(objective="binary", **params)

In [11]:
model.fit(X_train, y_train, eval_set=[(X_test, y_test)],
              eval_metric=['binary_logloss'],
              early_stopping_rounds=300, 
              #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
              verbose=1)

D:\IDE\anaconda3\envs\RL1\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
D:\IDE\anaconda3\envs\RL1\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\IDE\anaconda3\envs\RL1\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=37, min_child_samples=84 will be ignored. Current value: min_data_in_leaf=37
[1]	valid_0's binary_logloss: 0.692901
[2]	valid_0's binary_logloss: 0.683899
[3]	valid_0's binary_logloss: 0.673914
[4]	valid_0's binary_logloss: 0.664178
[5]	valid_0's binary_logloss: 0.65458
[6]	valid_0's binary_logloss: 0.645309
[7]	valid_0's binary_logloss: 0.63631
[8]	valid_0's binary_logloss: 0.627715
[9]	valid_0's binary_logloss: 0.619219
[10]	valid_0's binary_logloss: 0.611048
[11]	valid_0's binary_logloss: 0.603214
[12]	valid_0's binary_logloss: 0.595655
[13]	valid_0's binary_logloss: 0.588319
[14]	valid_0's binary_logloss: 0.581271
[15]	valid_0's binary_logloss: 0.574449
[16]	valid_0's binary_logloss: 0.567923
[17]	valid_0's binary_logloss: 0.561647
[18]	valid_0's binary_logloss: 0.55557
[19]	valid_0's binary_logloss: 0.549587
[20]	valid_0's binary_logloss: 0.543873
[21]	valid_0's binary_logloss: 0.538336
[22]	valid_0's binary_logloss: 0.532969
[23]	valid

[257]	valid_0's binary_logloss: 0.212476
[258]	valid_0's binary_logloss: 0.212193
[259]	valid_0's binary_logloss: 0.211931
[260]	valid_0's binary_logloss: 0.211578
[261]	valid_0's binary_logloss: 0.211006
[262]	valid_0's binary_logloss: 0.210661
[263]	valid_0's binary_logloss: 0.210363
[264]	valid_0's binary_logloss: 0.210119
[265]	valid_0's binary_logloss: 0.209854
[266]	valid_0's binary_logloss: 0.209532
[267]	valid_0's binary_logloss: 0.209268
[268]	valid_0's binary_logloss: 0.208722
[269]	valid_0's binary_logloss: 0.208238
[270]	valid_0's binary_logloss: 0.207932
[271]	valid_0's binary_logloss: 0.207665
[272]	valid_0's binary_logloss: 0.20737
[273]	valid_0's binary_logloss: 0.207071
[274]	valid_0's binary_logloss: 0.20689
[275]	valid_0's binary_logloss: 0.206657
[276]	valid_0's binary_logloss: 0.206357
[277]	valid_0's binary_logloss: 0.205875
[278]	valid_0's binary_logloss: 0.205543
[279]	valid_0's binary_logloss: 0.205193
[280]	valid_0's binary_logloss: 0.204827
[281]	valid_0's bi

[513]	valid_0's binary_logloss: 0.16195
[514]	valid_0's binary_logloss: 0.161913
[515]	valid_0's binary_logloss: 0.161941
[516]	valid_0's binary_logloss: 0.161914
[517]	valid_0's binary_logloss: 0.161829
[518]	valid_0's binary_logloss: 0.161744
[519]	valid_0's binary_logloss: 0.161662
[520]	valid_0's binary_logloss: 0.161635
[521]	valid_0's binary_logloss: 0.161646
[522]	valid_0's binary_logloss: 0.161568
[523]	valid_0's binary_logloss: 0.161488
[524]	valid_0's binary_logloss: 0.161471
[525]	valid_0's binary_logloss: 0.161499
[526]	valid_0's binary_logloss: 0.161378
[527]	valid_0's binary_logloss: 0.161253
[528]	valid_0's binary_logloss: 0.161197
[529]	valid_0's binary_logloss: 0.161014
[530]	valid_0's binary_logloss: 0.160912
[531]	valid_0's binary_logloss: 0.160837
[532]	valid_0's binary_logloss: 0.160758
[533]	valid_0's binary_logloss: 0.160702
[534]	valid_0's binary_logloss: 0.160631
[535]	valid_0's binary_logloss: 0.16052
[536]	valid_0's binary_logloss: 0.160428
[537]	valid_0's bi

[762]	valid_0's binary_logloss: 0.146692
[763]	valid_0's binary_logloss: 0.146687
[764]	valid_0's binary_logloss: 0.146583
[765]	valid_0's binary_logloss: 0.146566
[766]	valid_0's binary_logloss: 0.146559
[767]	valid_0's binary_logloss: 0.146535
[768]	valid_0's binary_logloss: 0.146482
[769]	valid_0's binary_logloss: 0.146487
[770]	valid_0's binary_logloss: 0.14639
[771]	valid_0's binary_logloss: 0.146337
[772]	valid_0's binary_logloss: 0.146309
[773]	valid_0's binary_logloss: 0.146233
[774]	valid_0's binary_logloss: 0.146153
[775]	valid_0's binary_logloss: 0.146153
[776]	valid_0's binary_logloss: 0.146141
[777]	valid_0's binary_logloss: 0.146113
[778]	valid_0's binary_logloss: 0.146102
[779]	valid_0's binary_logloss: 0.146038
[780]	valid_0's binary_logloss: 0.145965
[781]	valid_0's binary_logloss: 0.145892
[782]	valid_0's binary_logloss: 0.145837
[783]	valid_0's binary_logloss: 0.14584
[784]	valid_0's binary_logloss: 0.145772
[785]	valid_0's binary_logloss: 0.145719
[786]	valid_0's bi

...

[6577]	valid_0's binary_logloss: 0.0995537
[6578]	valid_0's binary_logloss: 0.0995526
[6579]	valid_0's binary_logloss: 0.0995447
[6580]	valid_0's binary_logloss: 0.0995428
[6581]	valid_0's binary_logloss: 0.0995434
[6582]	valid_0's binary_logloss: 0.099534
[6583]	valid_0's binary_logloss: 0.0995388
[6584]	valid_0's binary_logloss: 0.0995175
[6585]	valid_0's binary_logloss: 0.0995341
[6586]	valid_0's binary_logloss: 0.0995485
[6587]	valid_0's binary_logloss: 0.0995632
[6588]	valid_0's binary_logloss: 0.0995404
[6589]	valid_0's binary_logloss: 0.0995429
[6590]	valid_0's binary_logloss: 0.099556
[6591]	valid_0's binary_logloss: 0.0995483
[6592]	valid_0's binary_logloss: 0.0995319
[6593]	valid_0's binary_logloss: 0.0995291
[6594]	valid_0's binary_logloss: 0.0995093
[6595]	valid_0's binary_logloss: 0.0995054
[6596]	valid_0's binary_logloss: 0.0994991
[6597]	valid_0's binary_logloss: 0.0995128
[6598]	valid_0's binary_logloss: 0.0994884
[6599]	valid_0's binary_logloss: 0.0994768
[6600]	valid_

[6787]	valid_0's binary_logloss: 0.09931
[6788]	valid_0's binary_logloss: 0.0993114
[6789]	valid_0's binary_logloss: 0.0993089
[6790]	valid_0's binary_logloss: 0.0993125
[6791]	valid_0's binary_logloss: 0.0993165
[6792]	valid_0's binary_logloss: 0.0993109
[6793]	valid_0's binary_logloss: 0.0993169
[6794]	valid_0's binary_logloss: 0.0993148
[6795]	valid_0's binary_logloss: 0.0993124
[6796]	valid_0's binary_logloss: 0.099313
[6797]	valid_0's binary_logloss: 0.0993165
[6798]	valid_0's binary_logloss: 0.0993221
[6799]	valid_0's binary_logloss: 0.0993186
[6800]	valid_0's binary_logloss: 0.0993152
[6801]	valid_0's binary_logloss: 0.0993145
[6802]	valid_0's binary_logloss: 0.099318
[6803]	valid_0's binary_logloss: 0.0993184
[6804]	valid_0's binary_logloss: 0.0993128
[6805]	valid_0's binary_logloss: 0.0993178
[6806]	valid_0's binary_logloss: 0.0993154
[6807]	valid_0's binary_logloss: 0.0993214
[6808]	valid_0's binary_logloss: 0.0993253
[6809]	valid_0's binary_logloss: 0.0993239
[6810]	valid_0'

[6980]	valid_0's binary_logloss: 0.0994245
[6981]	valid_0's binary_logloss: 0.0994342
[6982]	valid_0's binary_logloss: 0.099434
[6983]	valid_0's binary_logloss: 0.0994436
[6984]	valid_0's binary_logloss: 0.0994401
[6985]	valid_0's binary_logloss: 0.0994455
[6986]	valid_0's binary_logloss: 0.0994409
[6987]	valid_0's binary_logloss: 0.0994244
[6988]	valid_0's binary_logloss: 0.0994313
[6989]	valid_0's binary_logloss: 0.0994324
[6990]	valid_0's binary_logloss: 0.0994386
[6991]	valid_0's binary_logloss: 0.0994335
[6992]	valid_0's binary_logloss: 0.0994213
[6993]	valid_0's binary_logloss: 0.0994114
[6994]	valid_0's binary_logloss: 0.0994197
[6995]	valid_0's binary_logloss: 0.0994467
[6996]	valid_0's binary_logloss: 0.0994287
[6997]	valid_0's binary_logloss: 0.099418
[6998]	valid_0's binary_logloss: 0.099429
[6999]	valid_0's binary_logloss: 0.0994276
[7000]	valid_0's binary_logloss: 0.0994335
[7001]	valid_0's binary_logloss: 0.0994209
[7002]	valid_0's binary_logloss: 0.0994056
[7003]	valid_0

[7175]	valid_0's binary_logloss: 0.0994871
[7176]	valid_0's binary_logloss: 0.0994844
[7177]	valid_0's binary_logloss: 0.0994897
[7178]	valid_0's binary_logloss: 0.0994826
[7179]	valid_0's binary_logloss: 0.0994923
[7180]	valid_0's binary_logloss: 0.0994868
[7181]	valid_0's binary_logloss: 0.0994964
[7182]	valid_0's binary_logloss: 0.0995129
[7183]	valid_0's binary_logloss: 0.0995222
[7184]	valid_0's binary_logloss: 0.0995318
[7185]	valid_0's binary_logloss: 0.0995201
[7186]	valid_0's binary_logloss: 0.0995078
[7187]	valid_0's binary_logloss: 0.0994857
[7188]	valid_0's binary_logloss: 0.0994781
[7189]	valid_0's binary_logloss: 0.099478
[7190]	valid_0's binary_logloss: 0.0994724
[7191]	valid_0's binary_logloss: 0.0994737
[7192]	valid_0's binary_logloss: 0.0994786
[7193]	valid_0's binary_logloss: 0.0994727
[7194]	valid_0's binary_logloss: 0.0994639
[7195]	valid_0's binary_logloss: 0.0994565
[7196]	valid_0's binary_logloss: 0.0994485
[7197]	valid_0's binary_logloss: 0.099471
[7198]	valid_

LGBMClassifier(cat_l2=0.001571519620397213, cat_smooth=58,
               colsample_bytree=0.8638322535165064, device='cpu',
               learning_rate=0.017764429656900442, max_bin=728, max_depth=62,
               metric='binary_logloss', min_child_samples=84,
               min_data_in_leaf=37, num_iterations=2000000, num_leaves=190,
               objective='binary', reg_alpha=0.0541168857042065,
               reg_lambda=19.708332913135084, subsample=0.9988361911191573,
               task='prediction', verbosity=-1)

In [12]:
test_pred = test.iloc[: , 1:]

In [19]:
y_test_pred = model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_test_pred)))

Model accuracy score: 0.9444


In [13]:
pred = model.predict(test_pred)

In [14]:
len(test['key'].values) == len(pred)

True

In [15]:
test.copy()
submiss = pd.DataFrame({'key':test['key'].values,'Result':pred})
submiss['Result'] = submiss['Result'].replace(0,-1)

In [16]:
submiss

,key,Result
0,21338,1
1,21339,-1
2,21340,-1
3,21341,-1
4,21342,1
...,...,...
2095,23433,1
2096,23434,1
2097,23435,-1
2098,23436,1


In [33]:
submiss.to_csv('submission1.csv', index=False)

In [21]:
model.save_model('Lightbgm_model.txt', num_iteration=model.best_iteration) 

AttributeError: 'LGBMClassifier' object has no attribute 'save_model'